### Bring Your Own Autogluon Container 

In [19]:
!cd ../ && ./build_and_push.sh

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  41.42MB
Step 1/11 : FROM python:3.7.6-slim-buster
 ---> 84de2ffd919d
Step 2/11 : COPY requirements.txt /opt/program/requirements.txt
 ---> Using cache
 ---> 5e218ddefc50
Step 3/11 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          nginx          ca-certificates          git          python3-dev          gcc     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 6a3b064a2c32
Step 4/11 : RUN pip install --no-cache-dir -r /opt/program/requirements.txt
 ---> Using cache
 ---> 663fbc8ce76a
Step 5/11 : RUN pip install --upgrade mxnet     && pip install autogluon
 ---> Using cache
 ---> 791595f2b584

### Introduction to BYOC - data & execution integration 

### Deploy training job 

In [20]:
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer, json_serializer


sess = sagemaker.Session()
role = get_execution_role()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

In [21]:
training_data = 's3://vpon-workshop-yianc/train.csv'
test_data = 's3://vpon-workshop-yianc/test.csv'

In [22]:
# image = 'autogluon-tabular-data'
image = '230755935769.dkr.ecr.us-east-1.amazonaws.com/autogluon-tabular-data'

sm_model = Estimator(image_name=image, 
                     role=role, 
                     train_instance_count=1, 
#                      train_instance_type='local',
                     train_instance_type='ml.c5.xlarge',
                     hyperparameters={'label-column': 'SalePrice'}
                    )

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [23]:
sm_model.fit({'training': training_data, 'testing': test_data})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-09-24 06:53:24 Starting - Starting the training job...
2020-09-24 06:53:26 Starting - Launching requested ML instances......
2020-09-24 06:54:39 Starting - Preparing the instances for training...
2020-09-24 06:55:21 Downloading - Downloading input data
2020-09-24 06:55:21 Training - Downloading the training image......
2020-09-24 06:56:20 Training - Training image download completed. Training in progress..Parameters: ['train.csv']
/opt/ml/input/data/training
        Id  MSSubClass MSZoning  ...  SaleType  SaleCondition SalePrice
0        1          60       RL  ...        WD         Normal    208500
1        2          20       RL  ...        WD         Normal    181500
2        3          60       RL  ...        WD         Normal    223500
3        4          70       RL  ...        WD        Abnorml    140000
4        5          60       RL  ...        WD         Normal    250000
...    ...         ...      ...  ...       ...            ...       ...
1455  1456          60      

### Deploy inference endpoint 

In [41]:
predictor = sm_model.deploy(1, 'ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
Using already existing model: autogluon-tabular-data-2020-09-24-06-53-24-386


-----------------!

In [42]:
predictor

In [43]:
import pandas as pd
import json
import io

In [44]:
df = pd.read_csv('s3://vpon-workshop-yianc/test.csv')

In [45]:
record = df.head(10).T.to_dict()[8]
payload = json.dumps(record)
print(payload) 
predictor.content_type = 'application/json'
price = predictor.predict(payload)
price



{"Id": 1469, "MSSubClass": 20, "MSZoning": "RL", "LotFrontage": 85.0, "LotArea": 10176, "Street": "Pave", "Alley": NaN, "LotShape": "Reg", "LandContour": "Lvl", "Utilities": "AllPub", "LotConfig": "Inside", "LandSlope": "Gtl", "Neighborhood": "Gilbert", "Condition1": "Norm", "Condition2": "Norm", "BldgType": "1Fam", "HouseStyle": "1Story", "OverallQual": 7, "OverallCond": 5, "YearBuilt": 1990, "YearRemodAdd": 1990, "RoofStyle": "Gable", "RoofMatl": "CompShg", "Exterior1st": "HdBoard", "Exterior2nd": "HdBoard", "MasVnrType": "None", "MasVnrArea": 0.0, "ExterQual": "TA", "ExterCond": "TA", "Foundation": "PConc", "BsmtQual": "Gd", "BsmtCond": "TA", "BsmtExposure": "Gd", "BsmtFinType1": "GLQ", "BsmtFinSF1": 637.0, "BsmtFinType2": "Unf", "BsmtFinSF2": 0.0, "BsmtUnfSF": 663.0, "TotalBsmtSF": 1300.0, "Heating": "GasA", "HeatingQC": "Gd", "CentralAir": "Y", "Electrical": "SBrkr", "1stFlrSF": 1341, "2ndFlrSF": 0, "LowQualFinSF": 0, "GrLivArea": 1341, "BsmtFullBath": 1.0, "BsmtHalfBath": 0.0, "F

b'181393.25\n'

In [46]:
predictor.delete_endpoint()